In [1]:
import cv2
import dlib
import face_recognition
import numpy as np
import pandas as pd
import os, csv, time
from datetime import datetime
from imutils.video import VideoStream
from IPython.display import display, clear_output


In [2]:
path = 'Images'
images = []
classNames = []

# Get list of valid image files only
valid_extensions = ('.jpg', '.jpeg', '.png')

myList = [f for f in os.listdir(path) if f.lower().endswith(valid_extensions)]
print("Found images:", myList)

for cl in myList:
    curImg = cv2.imread(os.path.join(path, cl))
    if curImg is None:
        print(f"⚠️ Could not read {cl}, skipping...")
        continue
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])

print("Loaded", len(images), "valid images.")

def findEncodings(images):
    encodeList = []
    for img, name in zip(images, classNames):
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encodings = face_recognition.face_encodings(img)
        if len(encodings) > 0:
            encodeList.append(encodings[0])
            print(f"✅ Encoded {name}")
        else:
            print(f"⚠️ No face found in {name}, skipping...")
    return encodeList

encodeListKnown = findEncodings(images)
print("\nEncoding Complete ✅")


Found images: ['Anav.jpg']
Loaded 1 valid images.
✅ Encoded Anav

Encoding Complete ✅


In [8]:
from imutils import face_utils
def eye_aspect_ratio(eye):
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])
    C = np.linalg.norm(eye[0] - eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat/shape_predictor_68_face_landmarks.dat")  # download below
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

EYE_AR_THRESH = 0.23  
EYE_AR_CONSEC_FRAMES = 3


In [15]:
def mark_attendance(name):
    try:
        name, uid = name_id.split("_")
    except:
        name, uid = name_id, "N/A"

    date_str = datetime.now().strftime('%Y-%m-%d')
    filename = f"Attendance_{date_str}.csv"

    if not os.path.exists(filename):
        with open(filename, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["Name", "Time"])

    with open(filename, 'r') as f:
        existing_entries = [line.split(',')[0].strip() for line in f.readlines()]

    if name not in existing_entries:
        with open(filename, 'a', newline='') as f:
            writer = csv.writer(f)
            now = datetime.now()
            timeStr = now.strftime('%H:%M:%S')
            writer.writerow([name, timeStr])
            print(f"🟢 Attendance marked for {name} at {timeStr}")


In [11]:
cap = cv2.VideoCapture(1)
blink_counter = 0
blink_detected = False


frame_count = 0
max_frames = 500  # Auto-stop after ~15–20 seconds

while True:
    success, frame = cap.read()
    if not success:
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

    # Face recognition
    face_locations = face_recognition.face_locations(rgb_small_frame)
    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

    for (face_encoding, face_location) in zip(face_encodings, face_locations):
        matches = face_recognition.compare_faces(encodeListKnown, face_encoding)
        face_distances = face_recognition.face_distance(encodeListKnown, face_encoding)
        best_match_index = np.argmin(face_distances)

        if matches[best_match_index]:
            name = classNames[best_match_index].upper()

            # Eye blink detection (liveness check)
            for rect in faces:
                shape = predictor(gray, rect)
                shape = face_utils.shape_to_np(shape)
                leftEye = shape[lStart:lEnd]
                rightEye = shape[rStart:rEnd]
                leftEAR = eye_aspect_ratio(leftEye)
                rightEAR = eye_aspect_ratio(rightEye)
                ear = (leftEAR + rightEAR) / 2.0

                if ear < EYE_AR_THRESH:
                    blink_counter += 1
                else:
                    if blink_counter >= EYE_AR_CONSEC_FRAMES:
                        blink_detected = True
                    blink_counter = 0

                # Draw eye contours
                for (x, y) in np.concatenate((leftEye, rightEye), axis=0):
                    cv2.circle(frame, (x, y), 2, (0, 255, 255), -1)

            y1, x2, y2, x1 = [v * 4 for v in face_location]
            color = (0, 255, 0) if blink_detected else (0, 0, 255)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, name, (x1 + 6, y2 + 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

            if blink_detected:
                markAttendance(name)
                cv2.putText(frame, "Attendance Marked ✅", (50, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                blink_detected = False

    cv2.imshow('Face & Blink Attendance System', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
    frame_count+=1
    if frame_count>max_frames:
        break
cap.release()
cv2.destroyAllWindows()
